<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/train/mountain/*`) and images of forests (`./data/train/forest/*`). Treat mountains as the positive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](https://github.com/scrunts23/DS-Unit-4-Sprint-3-Deep-Learning/blob/main/module2-convolutional-neural-networks/data/train/mountain/art1131.jpg?raw=1)|![](https://github.com/scrunts23/DS-Unit-4-Sprint-3-Deep-Learning/blob/main/module2-convolutional-neural-networks/data/validation/forest/cdmc317.jpg?raw=1)|

The problem is relatively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several different possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/train/mountain/*`) and images of forests (`./data/train/forest/*`). Treat mountains as the positive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pre-trained layers from resnet
4. Report your model's accuracy

## Load in Data

This surprisingly more difficult than it seems, because you are working with directories of images instead of a single file. This boiler plate will help you download a zipped version of the directory of images. The directory is organized into "train" and "validation" which you can use inside an `ImageGenerator` class to stream batches of images thru your model.  


### Download & Summarize the Data

This step is completed for you. Just run the cells and review the results. 

In [6]:
! git clone https://github.com/PalmerTurley34/DS-Unit-4-Sprint-3-Deep-Learning.git

Cloning into 'DS-Unit-4-Sprint-3-Deep-Learning'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 1118 (delta 0), reused 0 (delta 0), pack-reused 1115
Receiving objects: 100% (1118/1118), 100.17 MiB | 45.49 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [7]:
import tensorflow as tf
import os

PATH = '/content/DS-Unit-4-Sprint-3-Deep-Learning/module2-convolutional-neural-networks/data'

In [8]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [9]:
train_mountain_dir = os.path.join(train_dir, 'mountain')  # directory with our training cat pictures
train_forest_dir = os.path.join(train_dir, 'forest')  # directory with our training dog pictures
validation_mountain_dir = os.path.join(validation_dir, 'mountain')  # directory with our validation cat pictures
validation_forest_dir = os.path.join(validation_dir, 'forest')  # directory with our validation dog pictures

In [10]:
num_mountain_tr = len(os.listdir(train_mountain_dir))
num_forest_tr = len(os.listdir(train_forest_dir))

num_mountain_val = len(os.listdir(validation_mountain_dir))
num_forest_val = len(os.listdir(validation_forest_dir))

total_train = num_mountain_tr + num_forest_tr
total_val = num_mountain_val + num_forest_val

In [11]:
print('total training mountain images:', num_mountain_tr)
print('total training forest images:', num_forest_tr)

print('total validation mountain images:', num_mountain_val)
print('total validation forest images:', num_forest_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

total training mountain images: 253
total training forest images: 269
total validation mountain images: 124
total validation forest images: 61
--
Total training images: 522
Total validation images: 185


### Keras `ImageGenerator` to Process the Data

This step is completed for you, but please review the code. The `ImageGenerator` class reads in batches of data from a directory and pass them to the model one batch at a time. Just like large text files, this method is advantageous, because it stifles the need to load a bunch of images into memory. 

Check out the documentation for this class method: [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class). You'll expand it's use in the third assignment objective.

In [12]:
batch_size = 16
epochs = 50
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [14]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 520 images belonging to 2 classes.


In [15]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 182 images belonging to 2 classes.


## Instatiate Model

In [16]:
import numpy as np
 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API
 
resnet = ResNet50(weights='imagenet', include_top=False)

94773248/94765736 [==============================] - 1s 0us/step


In [17]:
for layer in resnet.layers:
    layer.trainable = False

In [18]:
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [19]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

## Fit Model

In [20]:
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/50
32/32 [==============================] - 3s 91ms/step - loss: 0.8797 - binary_accuracy: 0.4960 - val_loss: 0.6195 - val_binary_accuracy: 0.6705
Epoch 2/50
32/32 [==============================] - 2s 63ms/step - loss: 0.5945 - binary_accuracy: 0.7083 - val_loss: 0.5516 - val_binary_accuracy: 0.7898
Epoch 3/50
32/32 [==============================] - 2s 64ms/step - loss: 0.5320 - binary_accuracy: 0.8036 - val_loss: 0.4715 - val_binary_accuracy: 0.7898
Epoch 4/50
32/32 [==============================] - 2s 63ms/step - loss: 0.4805 - binary_accuracy: 0.7857 - val_loss: 0.4778 - val_binary_accuracy: 0.8239
Epoch 5/50
32/32 [==============================] - 2s 63ms/step - loss: 0.4281 - binary_accuracy: 0.8294 - val_loss: 0.4753 - val_binary_accuracy: 0.8239
Epoch 6/50
32/32 [==============================] - 2s 65ms/step - loss: 0.4660 - binary_accuracy: 0.7956 - val_loss: 0.4438 - val_binary_accuracy: 0.8352
Epoch 7/50
32/32 [==============================] - 2s 65ms/step - los

# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(16, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

In [29]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

In [30]:
model.fit(
    train_data_gen,
    epochs=50,
    validation_data=val_data_gen
)

Epoch 1/50
33/33 [==============================] - 2s 59ms/step - loss: 0.8421 - binary_accuracy: 0.6212 - val_loss: 0.5974 - val_binary_accuracy: 0.7253
Epoch 2/50
33/33 [==============================] - 2s 54ms/step - loss: 0.3631 - binary_accuracy: 0.8635 - val_loss: 0.6633 - val_binary_accuracy: 0.6154
Epoch 3/50
33/33 [==============================] - 2s 54ms/step - loss: 0.3269 - binary_accuracy: 0.8846 - val_loss: 0.3978 - val_binary_accuracy: 0.8022
Epoch 4/50
33/33 [==============================] - 2s 54ms/step - loss: 0.2353 - binary_accuracy: 0.9212 - val_loss: 0.2930 - val_binary_accuracy: 0.8791
Epoch 5/50
33/33 [==============================] - 2s 54ms/step - loss: 0.2352 - binary_accuracy: 0.9308 - val_loss: 0.2833 - val_binary_accuracy: 0.8571
Epoch 6/50
33/33 [==============================] - 2s 55ms/step - loss: 0.2557 - binary_accuracy: 0.9058 - val_loss: 0.4429 - val_binary_accuracy: 0.7637
Epoch 7/50
33/33 [==============================] - 2s 56ms/step - los

# Custom CNN Model with Image Manipulations

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Simply, you should be able to modify our image generator for the problem. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [31]:
train_image_generator = ImageDataGenerator(rescale=1./255, rotation_range=90, width_shift_range=.5, height_shift_range=.5)
validation_image_generator = ImageDataGenerator(rescale=1./255, rotation_range=90, width_shift_range=.5, height_shift_range=.5)

In [32]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 520 images belonging to 2 classes.


In [33]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 182 images belonging to 2 classes.


In [34]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

In [35]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

In [36]:
model.fit(
    train_data_gen,
    epochs=30,
    validation_data=val_data_gen
)

Epoch 1/30
33/33 [==============================] - 9s 267ms/step - loss: 0.5296 - binary_accuracy: 0.7962 - val_loss: 0.3389 - val_binary_accuracy: 0.8736
Epoch 2/30
33/33 [==============================] - 9s 270ms/step - loss: 0.3995 - binary_accuracy: 0.8173 - val_loss: 0.6994 - val_binary_accuracy: 0.7527
Epoch 3/30
33/33 [==============================] - 9s 264ms/step - loss: 0.4210 - binary_accuracy: 0.8712 - val_loss: 0.2410 - val_binary_accuracy: 0.9176
Epoch 4/30
33/33 [==============================] - 9s 262ms/step - loss: 0.3743 - binary_accuracy: 0.8462 - val_loss: 0.4007 - val_binary_accuracy: 0.8407
Epoch 5/30
33/33 [==============================] - 9s 263ms/step - loss: 0.3867 - binary_accuracy: 0.8500 - val_loss: 0.3241 - val_binary_accuracy: 0.8516
Epoch 6/30
33/33 [==============================] - 9s 264ms/step - loss: 0.3060 - binary_accuracy: 0.8731 - val_loss: 0.3282 - val_binary_accuracy: 0.8407
Epoch 7/30
33/33 [==============================] - 9s 260ms/ste

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language